In [ ]:
# !pip install -e ../package/
import tensorflow as tf
import numpy as np
from thesispack.models import miniGAEv1
from thesispack.datasets import GaeDataset
from thesispack.methods import history_figure
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
gaed = GaeDataset()

In [ ]:
for x, atld, a in gaed.train:
    A = a
    X = x
    Atld = atld

In [ ]:
w_p = float(A.shape[0]*A.shape[1] * A.shape[1] - tf.reduce_sum(A)) / tf.reduce_sum(A)
norm = A.shape[1] * A.shape[2] / float((A.shape[1] * A.shape[2] - tf.reduce_sum(A)) * 2)
gae = miniGAEv1(A.shape[1],w_p, norm=norm, optimizer="Adam", learning_rate=1e-2)

In [ ]:
gae.train(gaed.train,1000,history_learning_process=True)

In [ ]:
plt.imshow(tf.nn.sigmoid(gae([X,Atld])[0]).numpy()[0])

In [ ]:
def A2G(A):
    g = nx.Graph()
    a = np.where(A == 1)
    a = [[x, y] for x, y in zip(a[0], a[1])]
    g.add_edges_from(a)
    return g

In [ ]:
g = A2G(A[0])
nx.draw(g, with_labels=True,font_size=20,node_size=600)

In [ ]:
if any(gae.history.values()):
    history_figure(gae.history)

In [ ]:
plt.imshow(Atld[0],vmin=0,vmax=1)

In [ ]:
idx_save = "32"
idx_load = ""

In [ ]:
emb = gae.encoder(X,Atld)
emb_flat = tf.concat([*emb[:]],axis=0)
Labels = ['0', '1', '2', '3', '4', '5']
zero_embs_idx = tf.where(tf.reduce_sum(tf.cast(emb_flat == 0, tf.int16),axis=1)<emb_flat.shape[1])[:,0]
emb_flat = (emb_flat-tf.reduce_mean(emb_flat))/tf.math.reduce_std(emb_flat)
emb_flat = tf.nn.tanh(emb_flat)
emb_flat = tf.gather(emb_flat, zero_embs_idx)
emb_flat_dict = dict(zip(Labels,emb_flat.numpy()))

np.save("../data/gae/gae-node-embs{}".format(idx_save), emb_flat_dict)

In [ ]:
g_embs = tf.reduce_sum(emb,axis=1)
g_embs = (g_embs-tf.reduce_mean(g_embs))/tf.math.reduce_std(g_embs)
g_embs = tf.nn.tanh(g_embs)

In [ ]:
# emb_flat_dict = np.load("../data/gae/gae-node-embs{}.npy".format(idx_load), allow_pickle=True)[()]
# emb_flat = np.array([list(v) for v in emb_flat_dict.values()])

In [ ]:
pca = PCA(n_components=2)
pca_emb = pca.fit_transform(emb_flat)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(pca_emb[:,0], pca_emb[:,1],'o', label='node embeddings', markersize=15)

for (v,l) in zip(pca_emb, Labels):
    plt.text(v[0],v[1],l,fontsize=20)
plt.legend()
# plt.savefig('DataFigures/gae/node-embs.eps', format='eps')